# < 네이버 영화 리뷰 크롤링 >  __기생충_

## # 1단계: URL 접속하고 파싱 개체 만들기

In [ ]:
import requests # 페이지 요청 모듈
from bs4 import BeautifulSoup # 데이터 파싱할 수 있는 모듈

# 영화 리뷰 주소
url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=161967&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page="
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html,'html.parser')

## # 2단계: 페이지 개수 구하기

In [ ]:
import re

count = int(re.sub(",","",soup.select(".total > em")[0].get_text()))
if count%10 !=0:
    page = count//10 + 1
else:
    page = count//10
    print(page)

3629


## # 3단계: 자료구조를 선언하고 데이터 추가하기

In [ ]:
import time
from tqdm import tqdm

review = list()

for p in tqdm(range(1, page+1),desc='Page'):
    response = requests.get(url+str(p))
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    for i in range(10):
        date = re.sub("\.","-",soup.select(".score_reple > dl > dt > em:nth-of-type(2)")[i].get_text())
        user = soup.select(".score_reple > dl > dt > em > a > span")[i].get_text()
        comment = soup.select(".score_reple > p > span")[i].get_text().strip()
        score = soup.select(".star_score > em")[i].get_text()
        likes = soup.select("._sympathyButton > strong")[i].get_text()
        dislikes = soup.select("._notSympathyButton > strong")[i].get_text()
        tmp = [date,user,comment,score,likes,dislikes]
        if len(tmp)==6:
            review.append(tmp)
        else:
            review.append("에러","에러","에러","에러","에러","에러")

Page: 100%|██████████| 3629/3629 [11:33<00:00,  5.23it/s]


## # 4단계: 데이터 확인하기

In [ ]:
review[0], len(review)

(['2019-05-30 15:58',
  'brilliant_AKA_밝음(bril****)',
  '비에 젖지 않는 고급 장난감 텐트와, 비에 젖다 못해 잠겨버리는 반지하 가구',
  '10',
  '28292',
  '1485'],
 36290)

## # 5단계: 데이터 변환

In [ ]:
import pandas as pd

df_review = pd.DataFrame(review, columns = ['date','user','comment','score','likes','dislikes'])
df_review['date'] = pd.to_datetime(df_review['date'])
df_review.head()

,date,user,comment,score,likes,dislikes
0,2019-05-30 15:58:00,brilliant_AKA_밝음(bril****),"비에 젖지 않는 고급 장난감 텐트와, 비에 젖다 못해 잠겨버리는 반지하 가구",10,28292,1485
1,2019-05-30 12:25:00,김희정(priv****),최근 본 영화중 가장 충격적이었음... 근데 보니까 15세말고 19세 걸어야될것같던데..,10,18490,1560
2,2019-05-30 14:41:00,리오(papi****),"지하철이라는 단어가 언급되는 순간, 대다수의 관객은 자신이 어디에 이입할 지를 안다.",10,17140,1099
3,2019-05-30 16:28:00,멍뭉이(sumi****),전 가정부가 집 벨 누를 때 부터 이 영화는 장르가 바뀐다... 역대급 꿀잼영화,10,16716,925
4,2019-05-30 15:13:00,여행의 발견(riod****),황금종려상 수상작을 자막 없이 볼 수 있다는 것 자체로 좋다.,10,15732,794


## # 6단계: 데이터 저장

In [ ]:
from datetime import datetime

now = datetime.today()
str(now.strftime("%Y%m%d"))
df_review.to_excel("기생충_MovieReview_v1.xlsx", encoding = 'utf-8')